# 🌟 Effortlessly Manage Knowledge Base with WebUI and Python

In this guide, we'll show you how to programmatically upload and manage files in your Knowledge Base using WebUI's API. Plus, you’ll learn how to build an **auto-synced knowledge base** that detects file changes in your directory and syncs them. Let’s dive in! 🦾📂✨


---

## 📚 Step 1: Create Your Knowledge in WebUI

1️⃣ Open your WebUI and create a new Knowledge base.  
2️⃣ Add a name and description for your Knowledge.  
👏 Example: We'll call ours "Open WebUI Knowledge"!  

![image.png](./images/create-knowledge.png)


### ✏️ Note Your Knowledge ID  

![image.png](./images/knowledge-id.png)
Once created, you'll need the `knowledge_id`. For example:  

In [1]:
knowledge_id = "835121e1-d02d-4e13-b5bc-d54434f6c8b0"

## 🧰 Step 2: Set Up Your Environment

🔧 Import the necessary tools and set up environment variables in Python. You can use `.env` for storing your configuration like `WEBUI_URL`, `TOKEN`, and `MODEL`.


In [2]:
import os
import requests


try:
    from dotenv import load_dotenv

    load_dotenv(".env")
except ImportError:
    print("dotenv not installed, skipping...")


WEBUI_URL = os.getenv("WEBUI_URL", "http://localhost:8080") 
TOKEN = os.getenv("TOKEN", "your-webui-token-here")
MODEL = os.getenv("MODEL", "llama3.2:latest")

## 💬 Step 3: Chat with the Model (No Files Yet)

Use the `chat` function to interact with the model even without any files 🚫📄. Here's an example:


In [3]:
def chat(query, file_id=None, collection_id=None):
    url = f'{WEBUI_URL}/api/chat/completions'
    headers = {
        'Authorization': f'Bearer {TOKEN}',
        'Content-Type': 'application/json'
    }

    files = []
    if file_id:
        files.append({'type': 'file', 'id': file_id})

    if collection_id:
        files.append({'type': 'collection', 'id': collection_id})
        
    payload = {
        'model': MODEL,
        'messages': [{'role': 'user', 'content': query}],
        'files': files
    }
    response = requests.post(url, headers=headers, json=payload)
    return response.json()

👉 **Chat Example**  

In [4]:
response = chat("What is Open WebUI?")
print(response)

{'id': 'llama3.2:latest-fbc97f17-e228-4459-a6e3-0c918264172a', 'created': 1735372215, 'model': 'llama3.2:latest', 'choices': [{'index': 0, 'logprobs': None, 'finish_reason': 'stop', 'message': {'content': "OpenWebUI (OWUI) is an open-source web-based user interface that provides a simple and intuitive way to manage and monitor various Linux systems, including servers, virtual machines, and containers. It was created by Thomas Koenig and has been around since 2010.\n\nOpenWebUI offers a range of features, including:\n\n1. Console access: allows administrators to remotely access the console output of a system.\n2. File management: enables users to upload and download files between systems.\n3. Network monitoring: provides information about network connections and bandwidth usage.\n4. CPU and memory usage monitoring: displays real-time data on CPU and memory utilization.\n5. Disk space monitoring: shows available disk space and usage patterns.\n6. System information: provides detailed sys

## 📤 Step 4: Upload a File to WebUI

Let’s upload a file (e.g., `whitepaper.pdf`) to WebUI! 📄


In [5]:
def upload_file(file_path):
    url = f'{WEBUI_URL}/api/v1/files/'
    headers = {
        'Authorization': f'Bearer {TOKEN}',
        'Accept': 'application/json'
    }
    files = {'file': open(file_path, 'rb')}
    response = requests.post(url, headers=headers, files=files)
    return response.json()

In [6]:
uploaded_file = upload_file("./whitepaper.pdf")

👉 **Chat w/ File Example**  

In [7]:
response = chat("What is Open WebUI?",file_id=uploaded_file['id'])
print(response)

{'id': 'llama3.2:latest-e944bc15-dff8-439e-a22a-1e90d3e0060f', 'created': 1735372233, 'model': 'llama3.2:latest', 'choices': [{'index': 0, 'logprobs': None, 'finish_reason': 'stop', 'message': {'content': "Open WebUI is an open-source interface for local and/or private Large Language Models (LLMs). It offers a flexible way to interact with multiple LLMs in various configurations, enabling users to evaluate the models' performance across different settings and use cases. This flexibility allows for comparing different models under identical conditions or the same model under varying conditions, providing a rich dataset for nuanced analysis.\n\nThe primary strength of open-source UIs like Open WebUI lies in their ability to obtain data from real-world LLMs that perform on diverse hardware. It enables a more holistic analysis, aiding in the optimization of LLMs for a wide range of applications.\n\nOne notable advantage of Open WebUI is its streamlined data exportability, which significant

## 🧠 Step 5: Add a File to Your Knowledge

Now, let’s add your uploaded file to the knowledge base. 📂🔗

In [8]:
def add_file_to_knowledge(knowledge_id, file_id):
    url = f'{WEBUI_URL}/api/v1/knowledge/{knowledge_id}/file/add'
    headers = {
        'Authorization': f'Bearer {TOKEN}',
        'Content-Type': 'application/json'
    }
    data = {'file_id': file_id}
    response = requests.post(url, headers=headers, json=data)
    return response.json()

In [12]:
knowledge = add_file_to_knowledge(knowledge_id, uploaded_file['id'])

🎉 **Success**: Your file is now part of the knowledge base!  
Open your WebUI and refresh–you’ll see your file listed there. ✅🎊  
![Knowledge File Added Screenshot](./images/knowledge-with-file.png)


👉 **Chat w/ Collection Example**  

In [11]:
response = chat("What is Open WebUI?",collection_id=knowledge['id'])
print(response)

{'id': 'llama3.2:latest-52e675f8-2fc9-4961-8a92-0dec3d1a81ee', 'created': 1735372264, 'model': 'llama3.2:latest', 'choices': [{'index': 0, 'logprobs': None, 'finish_reason': 'stop', 'message': {'content': 'Open WebUI is an open-source Local Model UI (LMUI) that operates entirely locally, allowing users to interact with various Large Language Models (LLMs) in a similar experience to popular platforms like ChatGPT. This local deployment capability enables its use in a variety of settings, including high-security environments and remote locations with limited internet access [8].', 'role': 'assistant'}}], 'object': 'chat.completion'}


## ❌ Step 6: Remove a File from Knowledge (If Needed)

If you change your mind, removing a file from a knowledge base is just as easy! 🧹



In [13]:
def remove_file_from_knowledge(knowledge_id, file_id):
    url = f'{WEBUI_URL}/api/v1/knowledge/{knowledge_id}/file/remove'
    headers = {
        'Authorization': f'Bearer {TOKEN}',
        'Content-Type': 'application/json'
    }
    data = {'file_id': file_id}
    response = requests.post(url, headers=headers, json=data)
    return response.json()

In [14]:
response = remove_file_from_knowledge(knowledge_id, uploaded_file['id'])
print(response)

{'id': '835121e1-d02d-4e13-b5bc-d54434f6c8b0', 'user_id': 'a750f3d8-586a-48b9-b6d5-508463a18ad5', 'name': 'Open WebUI', 'description': 'Open WebUI Knowledge', 'data': {'file_ids': []}, 'meta': None, 'access_control': None, 'created_at': 1735371354, 'updated_at': 1735372303, 'files': []}


## 💡 Bonus: Build a Remotely Synced Knowledge Base 🤖📂

With these steps, you can create a **remotely synced knowledge base** that detects file changes in your local directories and syncs them to WebUI. 🚀 Here's how to get started:

1️⃣ Use the [Python `watchdog` library](https://pypi.org/project/watchdog/) to monitor changes in a directory.  
2️⃣ Detect new, updated, or deleted files.  
3️⃣ Automatically add/upload new files or remove deleted ones in the knowledge base.

### Example Idea

```python
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import time

class KnowledgeSync(FileSystemEventHandler):
    def __init__(self, knowledge_id):
        self.knowledge_id = knowledge_id

    def on_created(self, event):
        print(f'New file detected: {event.src_path}')
        file_info = upload_file(event.src_path)
        add_file_to_knowledge(self.knowledge_id, file_info['id'])

    def on_deleted(self, event):
        print(f'File deleted: {event.src_path}')
        # Logic to remove the file from knowledge

# Monitor directory
if __name__ == "__main__":
    path_to_watch = "./sync_directory"
    sync_handler = KnowledgeSync(knowledge_id)
    observer = Observer()
    observer.schedule(sync_handler, path_to_watch, recursive=True)
    observer.start()

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()
```

🛠️ With this script, you can sync files in real-time! Use it to **automate workflows**, keep remote knowledge bases up to date, or collaborate with your teammates. 🤝


## 📝 Summary 

Here’s what you’ve learned:
1. 🛠️ Set up your environment.  
2. 📤 Upload files to WebUI.  
3. 🔗 Add files to your Knowledge base.  
4. ❌ Easily remove files when needed.  
5. 🤖 BONUS: Build a synced knowledge base with Python!

👏 You're now a pro at managing files programmatically in WebUI! 🎉 Use this newfound power to automate your workflows and create smarter systems.

--- 

🚀 Go forth and build amazing things with your Knowledge base! 🦾✨  